# A. Polar dataframe

In [ ]:
!pip install polars
!pip install xlsx2csv
#polars-0.15.8
#xlsx2csv-0.8.0

In [99]:
import polars as pl
excel_file = "./existencias-colmenas.xlsx"
a = pl.read_excel(
      file=excel_file, 
      sheet_id=0, 
      xlsx2csv_options = {
          "linedelimitator":"\n"
      }, 
    )

print(a)

{'13': shape: (28, 7)
┌──────┬───────────────┬───────────────┬──────────────┬──────────────┬──────────────┬──────────────┐
│      ┆ 13.  Número   ┆ _duplicated_0 ┆ _duplicated_ ┆ _duplicated_ ┆ _duplicated_ ┆ _duplicated_ │
│ ---  ┆ de Unidades   ┆ ---           ┆ 1            ┆ 2            ┆ 3            ┆ 4            │
│ str  ┆ Producti...   ┆ str           ┆ ---          ┆ ---          ┆ ---          ┆ ---          │
│      ┆ ---           ┆               ┆ str          ┆ str          ┆ str          ┆ str          │
│      ┆ str           ┆               ┆              ┆              ┆              ┆              │
╞══════╪═══════════════╪═══════════════╪══════════════╪══════════════╪══════════════╪══════════════╡
│ null ┆ null          ┆ null          ┆ null         ┆ Tipo de      ┆ null         ┆ null         │
│      ┆               ┆               ┆              ┆ Colmenas     ┆              ┆              │
├╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌

## References

> [xlsx2csv](https://github.com/dilshod/xlsx2csv)

# C. Pandas dataframe

In [11]:
# import polars as pl
import pandas as pd

excel_file = "./existencias-colmenas.xlsx"

pandas_excel = pd.read_excel(excel_file,sheet_name=0, skiprows=2, )
print(pandas_excel.head())

   Unnamed: 0                    Unnamed: 1        Unnamed: 2  \
0         NaN                           NaN               NaN   
1         NaN       Total Nacional - Región             Total   
2         NaN                           NaN  Número de UPA3,4   
3         NaN                Total Nacional              5909   
4         NaN  Región de Arica y Parinacota                 6   

           Unnamed: 3           Unnamed: 4          Unnamed: 5  \
0                 NaN     Tipo de Colmenas                 NaN   
1                 NaN  Colmenas Modernas 5                 NaN   
2  Número de colmenas       Número de UPA4  Número de colmenas   
3              333037                 4997              306758   
4                  53                    5                  49   

            Unnamed: 6          Unnamed: 7  
0                  NaN                 NaN  
1  Colmenas Rústicas 6                 NaN  
2       Número de UPA4  Número de colmenas  
3                  995          

# B. PySpark dataframe

In [1]:
!pip install pyspark
!pip install findspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 281.4 MB 34 kB/s 
     |████████████████████████████████| 199 kB 38.3 MB/s 
  Created wheel for pyspark: filename=pyspark-3.3.1-py2.py3-none-any.whl size=281845512 sha256=714389ccc53ecead08614d3a902fa66dd48b42fd13da968c94d11c78b8b99c9b
  Stored in directory: /root/.cache/pip/wheels/43/dc/11/ec201cd671da62fa9c5cc77078235e40722170ceba231d7598
Successfully built pyspark
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


## Checking informatino session

In [ ]:
import pandas as pd
import pyspark

import findspark
findspark.init()

In [2]:
# Import SparkSession
from pyspark.sql import SparkSession
# Create a Spark Session
spark = SparkSession.builder.master("local[*]").getOrCreate()
# Check Spark Session Information
print(spark)

In [5]:
from pyspark import SparkContext
from pyspark.sql import SQLContext 

s_context = SparkContext.getOrCreate()
#s_context = SparkContext("local", "existencia_colmenas")
sql = SQLContext(s_context)

/usr/local/lib/python3.8/dist-packages/pyspark/sql/context.py:112: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  warnings.warn(


In [22]:
df = pd.read_excel("./existencias-colmenas.xlsx", sheet_name=0)
# Standardising all data types from pandas to String for spark
df = df.astype(str)

df_spark = sql.createDataFrame(df)


Unnamed: 0    float64
Unnamed: 1     object
Unnamed: 2     object
Unnamed: 3     object
Unnamed: 4     object
Unnamed: 5     object
Unnamed: 6     object
Unnamed: 7     object
dtype: object
<class 'pandas.core.frame.DataFrame'>
Unnamed: 0    object
Unnamed: 1    object
Unnamed: 2    object
Unnamed: 3    object
Unnamed: 4    object
Unnamed: 5    object
Unnamed: 6    object
Unnamed: 7    object
dtype: object


In [23]:
df_spark.show()

+----------+--------------------+----------------+------------------+-------------------+------------------+-------------------+------------------+
|Unnamed: 0|          Unnamed: 1|      Unnamed: 2|        Unnamed: 3|         Unnamed: 4|        Unnamed: 5|         Unnamed: 6|        Unnamed: 7|
+----------+--------------------+----------------+------------------+-------------------+------------------+-------------------+------------------+
|       nan|13.  Número de Un...|             nan|               nan|                nan|               nan|                nan|               nan|
|       nan|                 nan|             nan|               nan|                nan|               nan|                nan|               nan|
|       nan|                 nan|             nan|               nan|   Tipo de Colmenas|               nan|                nan|               nan|
|       nan|Total Nacional - ...|           Total|               nan|Colmenas Modernas 5|               nan|Colm

In [24]:
# Calculating Pi values using PySpark
import random
NUM_SAMPLES = 100000000
def inside(p):
  x, y = random.random(), random.random()
  return x*x + y*y < 1

count = s_context.parallelize(range(0, NUM_SAMPLES)).filter(inside).count()
pi = 4 * count / NUM_SAMPLES
print("Pi is roughly", pi)

Pi is roughly 3.14183284


## References




*   [Install PySpark 3 on Google Colab the Easy Way](https://medium.com/grabngoinfo/install-pyspark-3-on-google-colab-the-easy-way-577ec4a2bcd8)
*   [Introduction to PySpark](https://medium.com/the-researchers-guide/introduction-to-pyspark-a61f7217398e#:~:text=The%20first%20step%20would%20be,perform%20data%20loading%20and%20manipulations.&text=The%20next%20step%20is%20to,and%20instantiating%20with%20getOrCreate(%20).)
*   [How to Get Started with PySpark: compute power](https://towardsdatascience.com/how-to-get-started-with-pyspark-1adc142456ec?gi=10f32e5a2155)
*   [Dataframes and Big Data](https://medium.com/analytics-vidhya/introduction-to-pyspark-take-your-first-steps-into-big-data-analytics-b98cf7dce8b3)










## D. Cleaning messy data whyqd

In [6]:
!pip install whyqd
#whyqd-0.6.2 
#xlrd-2.0.1

In [ ]:
import whyqd
SCHEMA_SOURCE = "/full/path_to/2020_rates_data_schema.json"
DIRECTORY = "/path_to/working/directory/"
    # Note: these links may no longer work when you follow this tutorial. Get the latest ones...
INPUT_DATA = [
            "https://www.portsmouth.gov.uk/ext/documents-external/biz-ndr-properties-january-2020.xls",
            "https://www.portsmouth.gov.uk/ext/documents-external/biz-ndr-reliefs-january-2020.xls",
            "https://www.portsmouth.gov.uk/ext/documents-external/biz-empty-commercial-properties-january-2020.xls"]
method = whyqd.Method(directory=DIRECTORY, schema=SCHEMA)
method.add_data(source=INPUT_DATA)